#### Outline
- for each dataset: 
    - load dataset; 
    - for each network: 
        - load network
        - project 1000 test dataset samples
        - save to metric dataframe

In [1]:
# reload packages
%load_ext autoreload
%autoreload 2

### Choose GPU (this may not be needed on your computer)

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [3]:
import numpy as np
import pickle
import pandas as pd
import time
from umap import UMAP

In [4]:
from tfumap.umap import tfUMAP
import tensorflow as tf
from sklearn.decomposition import PCA
from openTSNE import TSNE

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [5]:
from tqdm.autonotebook import tqdm

In [6]:
from tfumap.paths import ensure_dir, MODEL_DIR, DATA_DIR

In [7]:
output_dir = MODEL_DIR/'projections' 

In [8]:
projection_speeds = pd.DataFrame(columns = ['method_', 'dimensions', 'dataset', 'speed'])

### MNIST

In [9]:
dataset = 'cifar10'
dims = (32,32,3)

##### load dataset

In [10]:
from tensorflow.keras.datasets import cifar10

# load dataset
(train_images, Y_train), (test_images, Y_test) = cifar10.load_data()
X_train = (train_images/255.).astype('float32')
X_test = (test_images/255.).astype('float32')
X_train = X_train.reshape((len(X_train), np.product(np.shape(X_train)[1:])))
X_test = X_test.reshape((len(X_test), np.product(np.shape(X_test)[1:])))

# subset a validation set
n_valid = 10000
X_valid = X_train[-n_valid:]
Y_valid = Y_train[-n_valid:].flatten()
X_train = X_train[:-n_valid]
Y_train = Y_train[:-n_valid].flatten()
Y_test = Y_test.flatten()

print(len(X_train), len(X_valid), len(X_test))

40000 10000 10000


In [11]:
X_test_flat = X_test

X_train_flat = X_train

#### Network 

##### 2 dims

In [12]:
load_loc = output_dir / dataset / 'network' 

In [13]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    batch_size = 100,
    dims = dims
)

In [14]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [15]:
n_repeats = 10
times = []
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    embedder.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    times.append(end_time - start_time)
    projection_speeds.loc[len(projection_speeds)] = ['network', 2, dataset, end_time - start_time]


seconds:  1.874784923857078
seconds:  0.3794771549291909
seconds:  0.3734123900067061
seconds:  0.3755004310514778
seconds:  0.3702197789680213
seconds:  0.378474015975371
seconds:  0.3569971020333469
seconds:  0.3871835151221603
seconds:  0.38585041696205735
seconds:  0.38425082084722817



In [16]:
z = embedder.transform(X_test_flat);
np.save( MODEL_DIR/'projections' / dataset / 'network' / 'z_test.npy', z)

In [17]:
##### Network CPU

with tf.device('/CPU:0'):
    n_repeats = 10
    times = []
    for i in tqdm(range(n_repeats)):
        start_time = time.monotonic()
        embedder.transform(X_test_flat);
        end_time = time.monotonic()
        print('seconds: ', end_time - start_time)
        times.append(end_time - start_time)
        projection_speeds.loc[len(projection_speeds)] = ['network-cpu', 2, dataset, end_time - start_time]

seconds:  0.692751414841041
seconds:  0.6400489979423583
seconds:  0.6192996150348336
seconds:  0.636889728019014
seconds:  0.6178402330260724
seconds:  0.64328903099522
seconds:  0.6315891661215574
seconds:  0.6589479590766132
seconds:  0.6676586179528385
seconds:  0.6246913690119982



##### 64 dims

In [18]:
load_loc = output_dir / dataset /"64"/ 'network' 

In [19]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    batch_size = 100,
    dims = dims
)

In [20]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [21]:
n_repeats = 10
times = []
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    embedder.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    times.append(end_time - start_time)
    projection_speeds.loc[len(projection_speeds)] = ['network', 64, dataset, end_time - start_time]

seconds:  0.435495647136122
seconds:  0.3813212178647518
seconds:  0.37471195799298584
seconds:  0.37979300506412983
seconds:  0.3835528821218759
seconds:  0.3817722708918154
seconds:  0.38441167515702546
seconds:  0.38366482499986887
seconds:  0.37157777906395495
seconds:  0.3721130739431828



In [22]:
z = embedder.transform(X_test_flat);
out = MODEL_DIR/'projections' / dataset / '64' / 'network' / 'z_test.npy'
np.save( out, z)

##### Network CPU

In [23]:
with tf.device('/CPU:0'):
    n_repeats = 10
    times = []
    for i in tqdm(range(n_repeats)):
        start_time = time.monotonic()
        embedder.transform(X_test_flat);
        end_time = time.monotonic()
        print('seconds: ', end_time - start_time)
        times.append(end_time - start_time)
        projection_speeds.loc[len(projection_speeds)] = ['network-cpu', 64, dataset, end_time - start_time]

seconds:  0.6672555468976498
seconds:  0.6753822390455753
seconds:  0.6320249699056149
seconds:  0.6222059191204607
seconds:  0.6180766997858882
seconds:  0.6158034859690815
seconds:  0.6382959592156112
seconds:  0.6408430230803788
seconds:  0.6283011038322002
seconds:  0.6170152500271797



### AE 

##### 2 dims

In [24]:
load_loc = output_dir / dataset / 'autoencoder' 

In [25]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    decoding_method = "autoencoder",
    batch_size = 100,
    dims = dims
)

In [26]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [27]:
decoder = tf.keras.models.load_model((load_loc / 'decoder').as_posix())
embedder.decoder = decoder

In [28]:
n_repeats = 10
times = []
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    embedder.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    times.append(end_time - start_time)
    projection_speeds.loc[len(projection_speeds)] = ['network', 2, dataset, end_time - start_time]


seconds:  0.41231335513293743
seconds:  0.3749568169005215
seconds:  0.36639093211852014
seconds:  0.3599984881002456
seconds:  0.3579307298641652
seconds:  0.3640342438593507
seconds:  0.37799659301526845
seconds:  0.38012760295532644
seconds:  0.3791631569620222
seconds:  0.3745101629756391



In [29]:
z = embedder.transform(X_test_flat);
np.save( MODEL_DIR/'projections' / dataset / 'autoencoder' / 'z_test.npy', z)

##### 64 dims

In [30]:
load_loc = output_dir / dataset /"64"/ 'autoencoder' 

In [31]:
embedder = tfUMAP(
    direct_embedding=False,
    verbose=True,
    negative_sample_rate=5,
    training_epochs=5,
    decoding_method = "autoencoder",
    batch_size = 100,
    dims = dims
)

In [32]:
encoder = tf.keras.models.load_model((load_loc / 'encoder').as_posix())
embedder.encoder = encoder

In [33]:
decoder = tf.keras.models.load_model((load_loc / 'decoder').as_posix())
embedder.decoder = decoder

In [34]:
n_repeats = 10
times = []
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    embedder.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    times.append(end_time - start_time)
    projection_speeds.loc[len(projection_speeds)] = ['network', 64, dataset, end_time - start_time]

seconds:  0.41536225308664143
seconds:  0.34870417579077184
seconds:  0.3780167440418154
seconds:  0.38507360592484474
seconds:  0.3804287030361593
seconds:  0.38184590404853225
seconds:  0.37666002521291375
seconds:  0.3794912868179381
seconds:  0.38970558787696064
seconds:  0.3865322871133685



In [35]:
z = embedder.transform(X_test_flat);
out = MODEL_DIR/'projections' / dataset / '64' / 'autoencoder' / 'z_test.npy'
np.save( out, z)

#### UMAP-learn

##### 2 dims

In [36]:
embedder = UMAP(n_components = 2, verbose=True)
z_umap = embedder.fit_transform(X_train_flat)

UMAP(dens_frac=0.0, dens_lambda=0.0, verbose=True)
Construct fuzzy simplicial set
Wed Jul 15 18:21:14 2020 Finding Nearest Neighbors
Wed Jul 15 18:21:14 2020 Building RP forest with 15 trees
Wed Jul 15 18:21:16 2020 parallel NN descent for 15 iterations
	 0  /  15
	 1  /  15
	 2  /  15
	 3  /  15
	 4  /  15
	 5  /  15
Wed Jul 15 18:21:28 2020 Finished Nearest Neighbor Search
Wed Jul 15 18:21:30 2020 Construct embedding
	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Wed Jul 15 18:21:58 2020 Finished embedding


In [37]:
n_repeats = 10
times = []
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    embedder.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    times.append(end_time - start_time)
    projection_speeds.loc[len(projection_speeds)] = ['umap-learn', 2, dataset, end_time - start_time]


	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  23.63913108711131
	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  12.671670091804117
	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  11.785255311988294
	completed  0  / 

In [38]:
out

PosixPath('/mnt/cube/tsainbur/Projects/github_repos/umap_tf_networks/models/projections/cifar10/64/autoencoder/z_test.npy')

In [39]:
z = embedder.transform(X_test_flat);
out = MODEL_DIR/'projections' / dataset / 'umap-learn' / 'z_test.npy'
ensure_dir(out)
np.save(out, z)

	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs


##### 64 dims

In [40]:
embedder = UMAP(n_components = 64, verbose=True)
z_umap = embedder.fit_transform(X_train_flat)

UMAP(dens_frac=0.0, dens_lambda=0.0, n_components=64, verbose=True)
Construct fuzzy simplicial set
Wed Jul 15 18:24:15 2020 Finding Nearest Neighbors
Wed Jul 15 18:24:15 2020 Building RP forest with 15 trees
Wed Jul 15 18:24:17 2020 parallel NN descent for 15 iterations
	 0  /  15
	 1  /  15
	 2  /  15
	 3  /  15
	 4  /  15
	 5  /  15
Wed Jul 15 18:24:20 2020 Finished Nearest Neighbor Search
Wed Jul 15 18:24:21 2020 Construct embedding
	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Wed Jul 15 18:24:54 2020 Finished embedding


In [41]:
n_repeats = 10
times = []
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    embedder.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    projection_speeds.loc[len(projection_speeds)] = ['umap-learn', 64, dataset, end_time - start_time]


	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  12.545422846917063
	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  11.244350783061236
	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
seconds:  10.764399299863726
	completed  0  /

In [42]:
z = embedder.transform(X_test_flat);
out = MODEL_DIR/'projections' / dataset / '64' / 'umap-learn' / 'z_test.npy'
ensure_dir(out)
np.save(out, z)

	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs


#### PCA

##### 2 dims

In [43]:
pca = PCA(n_components=2)
z = pca.fit_transform(X_train_flat)

In [44]:
n_repeats = 10
times = []
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    pca.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    times.append(end_time - start_time)
    projection_speeds.loc[len(projection_speeds)] = ['pca', 2, dataset, end_time - start_time]


seconds:  0.05264236801303923
seconds:  0.06321813096292317
seconds:  0.06407096493057907
seconds:  0.07108574593439698
seconds:  0.061707987915724516
seconds:  0.06539362319745123
seconds:  0.06713679293170571
seconds:  0.061284495051950216
seconds:  0.06372776487842202
seconds:  0.0635275999084115



In [45]:
z = pca.transform(X_test_flat);
out = MODEL_DIR/'projections' / dataset / 'PCA' / 'z_test.npy'
ensure_dir(out)
np.save(out, z)

##### 64 dims

In [46]:
pca = PCA(n_components=64)
z = pca.fit_transform(X_train_flat)

In [47]:
n_repeats = 10
times = []
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    pca.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    times.append(end_time - start_time)
    projection_speeds.loc[len(projection_speeds)] = ['pca', 64, dataset, end_time - start_time]


seconds:  0.06336713512428105
seconds:  0.07780653890222311
seconds:  0.07852321001701057
seconds:  0.07353859581053257
seconds:  0.0793257721234113
seconds:  0.07759653218090534
seconds:  0.076399568002671
seconds:  0.07381777395494282
seconds:  0.07876881584525108
seconds:  0.07799441507086158



In [48]:
z = pca.transform(X_test_flat);
out = MODEL_DIR/'projections' / dataset / "64" / 'PCA'  / 'z_test.npy'
ensure_dir(out)
np.save(out, z)

#### TSNE

##### 2 dims

In [49]:
tsne = TSNE(
    n_components = 2,
    n_jobs=32,
    verbose=True
)

In [50]:
embedding_train = tsne.fit(X_train_flat)

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


--------------------------------------------------------------------------------
TSNE(n_jobs=32, neighbors=None, verbose=True)
--------------------------------------------------------------------------------
===> Finding 90 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 82.68 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.37 seconds
===> Calculating PCA-based initialization...
   --> Time elapsed: 1.30 seconds
===> Running optimization with exaggeration=12.00, lr=3333.33 for 250 iterations...
Iteration   50, KL divergence 6.7705, 50 iterations in 2.1137 sec
Iteration  100, KL divergence 6.3256, 50 iterations in 3.5605 sec
Iteration  150, KL divergence 6.3298, 50 iterations in 5.8472 sec
Iteration  200, KL divergence 6.3539, 50 iterations in 10.0566 sec
Iteration  250, KL divergence 6.3440, 50 iterations in 7.0360 sec
   --> Time elapsed: 28.62 seconds
===> Running optimization with exaggeration=1.00, lr=3333.33 for 

In [51]:
n_repeats = 10
times = []
for i in tqdm(range(n_repeats)):
    start_time = time.monotonic()
    embedding_train.transform(X_test_flat);
    end_time = time.monotonic()
    print('seconds: ', end_time - start_time)
    times.append(end_time - start_time)
    projection_speeds.loc[len(projection_speeds)] = ['TSNE', 2, dataset, end_time - start_time]

===> Finding 15 nearest neighbors in existing embedding using Annoy approximate search...
   --> Time elapsed: 10.55 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.03 seconds
===> Running optimization with exaggeration=4.00, lr=0.10 for 0 iterations...
   --> Time elapsed: 0.00 seconds
===> Running optimization with exaggeration=1.50, lr=0.10 for 250 iterations...
Iteration   50, KL divergence 193568.7237, 50 iterations in 0.2706 sec
Iteration  100, KL divergence 193530.0377, 50 iterations in 0.2779 sec
Iteration  150, KL divergence 193524.6180, 50 iterations in 0.2689 sec
Iteration  200, KL divergence 193529.3813, 50 iterations in 0.2908 sec
Iteration  250, KL divergence 193539.1925, 50 iterations in 0.2672 sec
   --> Time elapsed: 1.38 seconds
seconds:  12.179296564077958
===> Finding 15 nearest neighbors in existing embedding using Annoy approximate search...
   --> Time elapsed: 10.55 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.02 seconds


In [52]:
z = embedding_train.transform(X_test_flat);
out = MODEL_DIR/'projections' / dataset / 'TSNE' / 'z_test.npy'
ensure_dir(out)
np.save(out, z)

===> Finding 15 nearest neighbors in existing embedding using Annoy approximate search...
   --> Time elapsed: 10.33 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.03 seconds
===> Running optimization with exaggeration=4.00, lr=0.10 for 0 iterations...
   --> Time elapsed: 0.00 seconds
===> Running optimization with exaggeration=1.50, lr=0.10 for 250 iterations...
Iteration   50, KL divergence 193568.7237, 50 iterations in 0.2933 sec
Iteration  100, KL divergence 193530.0377, 50 iterations in 0.2967 sec
Iteration  150, KL divergence 193524.6180, 50 iterations in 0.3119 sec
Iteration  200, KL divergence 193529.3813, 50 iterations in 0.2506 sec
Iteration  250, KL divergence 193539.1925, 50 iterations in 0.2861 sec
   --> Time elapsed: 1.44 seconds


In [53]:
projection_speeds

,method_,dimensions,dataset,speed
0,network,2,cifar10,1.874785
1,network,2,cifar10,0.379477
2,network,2,cifar10,0.373412
3,network,2,cifar10,0.375500
4,network,2,cifar10,0.370220
...,...,...,...,...
105,TSNE,2,cifar10,11.949304
106,TSNE,2,cifar10,11.923504
107,TSNE,2,cifar10,11.913525
108,TSNE,2,cifar10,12.030830


### Save

In [54]:
save_loc = DATA_DIR / 'projection_speeds' / (dataset + '.pickle')
ensure_dir(save_loc)
projection_speeds.to_pickle(save_loc)